# Задача. Зробити модель, яка автоматично буде знаходити негативні коментарі.



## Інпут даних та огляд

In [1]:
# Зробимо окрему папку для наших даних
import os
data_path = 'C:/Users/Serhii/Downloads/Lec_24/data'
#os.makedirs(data_path)

In [2]:
# Завантажимо всі наші дані
import pandas as pd

train_path = f'{data_path}/jigsaw-toxic-comment-train.csv'
val_path = f'{data_path}/validation.csv'

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)

In [3]:
display(train_df.info())
print('-'*100)
display(train_df.describe())
print('-'*100)
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             223549 non-null  object
 1   comment_text   223549 non-null  object
 2   toxic          223549 non-null  int64 
 3   severe_toxic   223549 non-null  int64 
 4   obscene        223549 non-null  int64 
 5   threat         223549 non-null  int64 
 6   insult         223549 non-null  int64 
 7   identity_hate  223549 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 13.6+ MB


None

----------------------------------------------------------------------------------------------------


,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,223549.000000,223549.000000,223549.000000,223549.000000,223549.000000,223549.000000
mean,0.095657,0.008777,0.054306,0.003082,0.050566,0.009470
std,0.294121,0.093272,0.226621,0.055431,0.219110,0.096852
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


----------------------------------------------------------------------------------------------------


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train_df[train_df['toxic'] == 1].iloc[0,1]

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [7]:
train_df.comment_text[2]

"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."

In [8]:
display(val_df.info())
display(val_df.describe())
val_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            8000 non-null   int64 
 1   comment_text  8000 non-null   object
 2   lang          8000 non-null   object
 3   toxic         8000 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 250.1+ KB


None

,id,toxic
count,8000.00000,8000.000000
mean,3999.50000,0.153750
std,2309.54541,0.360731
min,0.00000,0.000000
25%,1999.75000,0.000000
50%,3999.50000,0.000000
75%,5999.25000,0.000000
max,7999.00000,1.000000


,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


In [9]:
from sklearn import metrics

def roc_auc(predictions,target):
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

## Процесінг даних

In [10]:
train_df.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True) # Видаляємо непотрібні колонки

sample_part = 12000
train_df = train_df.loc[:sample_part,:] # Візьмемо частину даних Щоб зробити тренування швидше

max_words = train_df['comment_text'].apply(lambda x:len(str(x).split())).max() # Максимальна кількість слів презентована у коментарі
print(f'max_words is {max_words}')

# Розбиваємо на x|y частини для комфортного використання.
X_train = train_df.comment_text
y_train = train_df.toxic

X_val = val_df.comment_text
y_val = val_df.toxic

max_words is 1403


In [11]:
# Токенізуємо слова
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import text

token = text.Tokenizer(num_words=None)
max_len = max_words + 100

# translation
# regex

token.fit_on_texts(list(X_train))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_val)

# Додаємо нулі до пропусків (zero padding)
xtrain_pad = utils.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = utils.pad_sequences(xvalid_seq, maxlen=max_len)

# Отримуємо словник слів
word_index = token.word_index

In [10]:
print('Оригінальний коментар:\n', X_train[0])
print('-'*50)
print('Послідовність:\n', xtrain_seq[0])
print('-'*50)
print('Padding:\n', xtrain_pad[0])
print('-'*50)

Оригінальний коментар:
 Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
--------------------------------------------------
Послідовність:
 [700, 73, 1, 136, 118, 174, 28, 561, 4328, 11057, 1181, 82, 340, 52, 2422, 16020, 50, 6350, 15, 62, 4685, 147, 5, 4025, 34, 120, 1258, 8709, 2725, 3, 51, 64, 267, 1, 385, 33, 1, 39, 27, 135, 70, 4026, 88, 3315, 3745, 2726, 1054]
--------------------------------------------------
Padding:
 [   0    0    0 ... 3745 2726 1054]
--------------------------------------------------


In [11]:
from itertools import islice

print('Словник слів:\n', list(islice(word_index.items(), 5)))

Словник слів:
 [('the', 1), ('to', 2), ('and', 3), ('of', 4), ('i', 5)]


## Моделювання

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model_v1 = Sequential()
model_v1.add(Embedding(len(word_index) + 1,
                  300,
                  input_length=max_len))
model_v1.add(SimpleRNN(100))
model_v1.add(Dense(1, activation='sigmoid'))

model_v1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_v1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1503, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_v1.fit(xtrain_pad, y_train,
          epochs=5,
          batch_size=256,
          validation_data = (xvalid_pad, y_val),
          )

Epoch 1/5
47/47 [==============================] - 89s 2s/step - loss: 0.3299 - accuracy: 0.8977 - val_loss: 0.4305 - val_accuracy: 0.8462
Epoch 2/5
47/47 [==============================] - 89s 2s/step - loss: 0.2329 - accuracy: 0.9169 - val_loss: 0.4943 - val_accuracy: 0.8469
Epoch 3/5
47/47 [==============================] - 90s 2s/step - loss: 0.0763 - accuracy: 0.9751 - val_loss: 0.5606 - val_accuracy: 0.8422
Epoch 4/5
47/47 [==============================] - 90s 2s/step - loss: 0.0126 - accuracy: 0.9979 - val_loss: 0.7081 - val_accuracy: 0.8447
Epoch 5/5
47/47 [==============================] - 91s 2s/step - loss: 0.0090 - accuracy: 0.9981 - val_loss: 0.6559 - val_accuracy: 0.8077


## Тестування та збереження

In [14]:
scores = model_v1.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores, y_val)))

250/250 [==============================] - 73s 290ms/step
Auc: 0.54%


In [15]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores, y_val)})

## Інші варіації

### Embedings

In [16]:
import numpy as np
from tqdm import tqdm

embeddings_index = {}
f = open(f'{data_path}/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [02:41, 13571.30it/s]

Found 2196017 word vectors.


In [17]:
# створимо матрицю вбудовування для слів, які є в наборі даних
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|████████████████████████████████████████████████████████████████████████| 43496/43496 [00:00<00:00, 374235.77it/s]


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model_v2 = Sequential()
model_v2.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False
                    ))
model_v2.add(SimpleRNN(100))
model_v2.add(Dense(1, activation='sigmoid'))

model_v2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_v2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1503, 300)         13049100  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 40,201
Non-trainable params: 13,049,100
_________________________________________________________________


In [19]:
model_v2.fit(xtrain_pad, y_train,
          epochs=5,
          batch_size=256,
          validation_data = (xvalid_pad, y_val),
          )

Epoch 1/5
47/47 [==============================] - 83s 2s/step - loss: 0.2619 - accuracy: 0.9112 - val_loss: 0.5049 - val_accuracy: 0.8405
Epoch 2/5
47/47 [==============================] - 79s 2s/step - loss: 0.1590 - accuracy: 0.9442 - val_loss: 0.4788 - val_accuracy: 0.8257
Epoch 3/5
47/47 [==============================] - 79s 2s/step - loss: 0.1690 - accuracy: 0.9469 - val_loss: 0.7608 - val_accuracy: 0.6344
Epoch 4/5
47/47 [==============================] - 76s 2s/step - loss: 0.2221 - accuracy: 0.9249 - val_loss: 0.5221 - val_accuracy: 0.8242
Epoch 5/5
47/47 [==============================] - 76s 2s/step - loss: 0.2199 - accuracy: 0.9235 - val_loss: 0.4919 - val_accuracy: 0.8246


In [20]:
scores = model_v2.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores, y_val)))
scores_model.append({'Model': 'Embeding','AUC_Score': roc_auc(scores, y_val)})

250/250 [==============================] - 71s 284ms/step
Auc: 0.56%


### LSTM

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model_v3 = Sequential()
model_v3.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False
                    ))
model_v3.add(LSTM(50))
model_v3.add(Dense(1, activation='sigmoid'))

model_v3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_v3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1503, 300)         13049100  
                                                                 
 lstm (LSTM)                 (None, 50)                70200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 13,119,351
Trainable params: 70,251
Non-trainable params: 13,049,100
_________________________________________________________________


In [22]:
model_v3.fit(xtrain_pad, y_train,
          epochs=5,
          batch_size=256,
          validation_data = (xvalid_pad, y_val),
          )

Epoch 1/5
47/47 [==============================] - 12s 223ms/step - loss: 0.3012 - accuracy: 0.8993 - val_loss: 0.4688 - val_accuracy: 0.8366
Epoch 2/5
47/47 [==============================] - 10s 214ms/step - loss: 0.1727 - accuracy: 0.9373 - val_loss: 0.4642 - val_accuracy: 0.7850
Epoch 3/5
47/47 [==============================] - 10s 216ms/step - loss: 0.1286 - accuracy: 0.9523 - val_loss: 0.5804 - val_accuracy: 0.6950
Epoch 4/5
47/47 [==============================] - 10s 215ms/step - loss: 0.1192 - accuracy: 0.9563 - val_loss: 0.5733 - val_accuracy: 0.7279
Epoch 5/5
47/47 [==============================] - 10s 216ms/step - loss: 0.1082 - accuracy: 0.9612 - val_loss: 0.5777 - val_accuracy: 0.7779


In [23]:
scores = model_v3.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores, y_val)))
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores, y_val)})

250/250 [==============================] - 7s 28ms/step
Auc: 0.61%


### GRU

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model_v4 = Sequential()
model_v4.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False
                    ))
model_v4.add(GRU(50))
model_v4.add(Dense(1, activation='sigmoid'))

model_v4.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_v4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1503, 300)         13049100  
                                                                 
 gru (GRU)                   (None, 50)                52800     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 13,101,951
Trainable params: 52,851
Non-trainable params: 13,049,100
_________________________________________________________________


In [25]:
model_v4.fit(xtrain_pad, y_train,
          epochs=5,
          batch_size=128,
          validation_data = (xvalid_pad, y_val),
          )

Epoch 1/5
94/94 [==============================] - 13s 120ms/step - loss: 0.2552 - accuracy: 0.9133 - val_loss: 0.5626 - val_accuracy: 0.8443
Epoch 2/5
94/94 [==============================] - 11s 116ms/step - loss: 0.1277 - accuracy: 0.9516 - val_loss: 0.4801 - val_accuracy: 0.8075
Epoch 3/5
94/94 [==============================] - 11s 120ms/step - loss: 0.1090 - accuracy: 0.9594 - val_loss: 0.4701 - val_accuracy: 0.8066
Epoch 4/5
94/94 [==============================] - 11s 122ms/step - loss: 0.0966 - accuracy: 0.9646 - val_loss: 0.5109 - val_accuracy: 0.7885
Epoch 5/5
94/94 [==============================] - 11s 117ms/step - loss: 0.0876 - accuracy: 0.9674 - val_loss: 0.5504 - val_accuracy: 0.7574


In [26]:
scores = model_v4.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores, y_val)))
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores, y_val)})

250/250 [==============================] - 7s 26ms/step
Auc: 0.64%


In [27]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.5356438015635696},
 {'Model': 'Embeding', 'AUC_Score': 0.5593912046210566},
 {'Model': 'LSTM', 'AUC_Score': 0.6147155071993851},
 {'Model': 'GRU', 'AUC_Score': 0.6398685616841397}]

### Bi-Directional

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

model_v5 = Sequential()
model_v5.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False
                    ))
model_v5.add(Bidirectional(GRU(50)))
model_v5.add(Dense(1, activation='sigmoid'))

model_v5.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_v5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1503, 300)         13049100  
                                                                 
 bidirectional (Bidirectiona  (None, 200)              241200    
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 13,290,501
Trainable params: 241,401
Non-trainable params: 13,049,100
_________________________________________________________________


In [29]:
model_v5.fit(xtrain_pad, y_train,
          epochs=5,
          batch_size=128,
          validation_data = (xvalid_pad, y_val),
          )

Epoch 1/5
94/94 [==============================] - 37s 367ms/step - loss: 0.2315 - accuracy: 0.9163 - val_loss: 0.4469 - val_accuracy: 0.8300
Epoch 2/5
94/94 [==============================] - 33s 356ms/step - loss: 0.1303 - accuracy: 0.9546 - val_loss: 0.5508 - val_accuracy: 0.8432
Epoch 3/5
94/94 [==============================] - 34s 359ms/step - loss: 0.1045 - accuracy: 0.9611 - val_loss: 0.5535 - val_accuracy: 0.8395
Epoch 4/5
94/94 [==============================] - 33s 354ms/step - loss: 0.0946 - accuracy: 0.9637 - val_loss: 0.5147 - val_accuracy: 0.8254
Epoch 5/5
94/94 [==============================] - 32s 340ms/step - loss: 0.0836 - accuracy: 0.9680 - val_loss: 0.6559 - val_accuracy: 0.8443


In [30]:
scores = model_v5.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores, y_val)))
scores_model.append({'Model': 'BI-DIR','AUC_Score': roc_auc(scores, y_val)})

250/250 [==============================] - 17s 66ms/step
Auc: 0.62%


In [31]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.5356438015635696},
 {'Model': 'Embeding', 'AUC_Score': 0.5593912046210566},
 {'Model': 'LSTM', 'AUC_Score': 0.6147155071993851},
 {'Model': 'GRU', 'AUC_Score': 0.6398685616841397},
 {'Model': 'BI-DIR', 'AUC_Score': 0.623741758835609}]

In [32]:
path_to_save = 'C:/Users/Serhii/Downloads/Lec_24/'
model_v5.save(f'{path_to_save}/model')

INFO:tensorflow:Assets written to: C:/Users/Serhii/Downloads/Lec_24//model\assets


INFO:tensorflow:Assets written to: C:/Users/Serhii/Downloads/Lec_24//model\assets


### Results

In [33]:
from tensorflow.keras.models import load_model

# download model
model_loaded = load_model(f'{path_to_save}/model')

In [34]:
max_len = max_words + 100
# download X
test_path = f'{data_path}/test.csv'
test_df = pd.read_csv(test_path)
X_test = test_df.content
X_test_seq = token.texts_to_sequences(X_test)
X_test_pad = utils.pad_sequences(X_test_seq, maxlen=max_len)

# download y
test_labels = f'{data_path}/test_labels.csv'
test_labels = pd.read_csv(test_labels)
y_test = test_labels.toxic

In [37]:
test_df

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
...,...,...,...
63807,63807,"No, non risponderò, come preannunciato. Prefer...",it
63808,63808,"Ciao, I tecnici della Wikimedia Foundation sta...",it
63809,63809,innnazitutto ti ringrazio per i ringraziamenti...,it
63810,63810,Kaç olumlu oy gerekiyor? Şu an 7 oldu. Hayır...,tr


In [35]:
# Final check
test_loss, test_acc = model_loaded.evaluate(X_test_pad, y_test)
print(f'loss = {test_loss}; accuracy = {test_acc}')

1995/1995 [==============================] - 139s 69ms/step - loss: 1.0438 - accuracy: 0.7741
loss = 1.0438292026519775; accuracy = 0.7741490602493286


In [36]:
test_labels['toxic_pred'] = model_loaded.predict(X_test_pad, verbose=1) # Write results
print('AUC_Score:', roc_auc(test_labels['toxic_pred'] , y_test))

1995/1995 [==============================] - 131s 65ms/step
AUC_Score: 0.5707888371291219


In [ ]:
test_labels.to_csv('results.csv', index=False) # save

### Як використати BERT?